# Hypergraph exploration

The goal of this notebook is to use the tools in the [Tutte Institute ``thisnotthat`` library](https://github.com/TutteInstitute/thisnotthat) to construct hypergragh embeddings where we jointly embed vertices and hyperedges.

We will make use of a recipe dataset. After filtering out recipes having two ingredients or less (see data-setup), the data consists of 39,559 recipes (hyperedges) and 6,714 ingredients (vertices). The largest recipe has 65 ingredients (must be good!). Each recipe is assigned to a country (edge label), with 20 countries total. The data and some work done with it can be found here:

* https://arxiv.org/pdf/1910.09943.pdf
* https://www.cs.cornell.edu/~arb/data/cat-edge-Cooking/

### Setup

To create the environment:

* conda create -n tnt_simple numba datashader jupyter ipykernel
* pip install thisnotthat seaborn

In [10]:
import thisnotthat as tnt
import panel as pn

import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import csv

In [11]:
import scipy.sparse
import vectorizers
import vectorizers.transformers
import umap
from scipy.sparse import vstack
import warnings
      
warnings.simplefilter("ignore")
sns.set()

# Data preparation

We will make use of the recipe dataset. It consists of 39,774 recipes (hyperedges) that are sets of vertices (6,714 ingredients total). The largest recipe has 65 ingredients (must be good!). Each recipe is assigned to a country (edge label), 20 countries total. The data and some work done with it can be found here:

This function 
* reads the data
* keeps only the recipes containing at least 3 ingredients (after this pruning we are left with 6,714 ingredients and 39,559 recipes)
* chooses a country color mapping that respects countries' proximities, or continent - nearby countries are assigned to similar colors. This is to help with the eye-ball evaluation of the visualization and make it more pleasant.

In [12]:
def read_format_recipes(recipe_min_size=3):
    ingredients_id = pd.read_csv('../data/cat-edge-Cooking/node-labels.txt', sep='\t', header=None)
    ingredients_id.index = [x+1 for x in ingredients_id.index]
    ingredients_id.columns = ['Ingredient']
    
    recipes_with_id = []
    with open('../data/cat-edge-Cooking/hyperedges.txt', newline = '') as hyperedges:
        hyperedge_reader = csv.reader(hyperedges, delimiter='\t')
        for hyperedge in hyperedge_reader:
            recipes_with_id.append(hyperedge)
            
    recipes_all = [[ingredients_id.loc[int(i)]['Ingredient'] for i in x] for x in recipes_with_id]
    
    # Keep recipes with 3 ingredients and more
    keep_recipes = np.where(np.array([len(x) for x in recipes_all])>=recipe_min_size)[0]
    recipes = [recipes_all[i] for i in keep_recipes]
    
    recipes_label_id_all = pd.read_csv('../data/cat-edge-Cooking/hyperedge-labels.txt', sep='\t', header=None)
    recipes_label_id_all.columns = ['label']
    recipes_label_id = recipes_label_id_all.iloc[keep_recipes].reset_index()

    label_name = pd.read_csv('../data/cat-edge-Cooking/hyperedge-label-identities.txt', sep='\t', header=None)
    label_name.columns = ['country']
    label_name.index = [x+1 for x in label_name.index]
    
    grps_tmp = {
        'asian' : ('chinese', 'filipino', 'japanese','korean', 'thai', 'vietnamese'),
        'american' : ('brazilian', 'mexican', 'southern_us'),
        'english' : ('british', 'irish'),
        'islands' : ('cajun_creole', 'jamaican'),
        'europe' : ('french', 'italian', 'spanish'),
        'others' : ('greek', 'indian', 'moroccan', 'russian')
    }

    grps = {key:[key+'.'+x for x in value] for key, value in grps_tmp.items()}


    color_key = {}
    for l, c in zip(grps['asian'], sns.color_palette("Blues", 6)[0:]):
        color_key[l] = matplotlib.colors.rgb2hex(c)
    for l, c in zip(grps['american'], sns.color_palette("Purples", 4)[1:]):
        color_key[l] = matplotlib.colors.rgb2hex(c)
    for l, c in zip(grps['others'], sns.color_palette("YlOrRd", 4)):
        color_key[l] = matplotlib.colors.rgb2hex(c)
    for l, c in zip(grps['europe'], sns.color_palette("light:teal", 4)[1:]):
        color_key[l] = matplotlib.colors.rgb2hex(c)
    for l, c in zip(grps['islands'], sns.color_palette("light:#660033", 4)[1:3]):
        color_key[l] = matplotlib.colors.rgb2hex(c)
    for l, c in zip(grps['english'], sns.color_palette("YlGn", 4)[1:]):
        color_key[l] = matplotlib.colors.rgb2hex(c)
    color_key["ingredient"] = "#777777bb"
    
    new_names = []
    for key, value in grps.items():
        new_names = new_names + value

    label_name['new_label'] = [new_name for x in label_name.country for new_name in new_names if x in new_name]
    
    return(recipes, recipes_label_id, ingredients_id, label_name, color_key)

In [13]:
# execfile('./00-recipes-setup.py')
recipes, recipes_label_id, ingredients_id, label_name, color_key = read_format_recipes()
recipes_label = [label_name.loc[i]['new_label'] for i in recipes_label_id.label]

# Build vertex (ingredient) vectors

We first build a vector representation of ingredients based on co-occurrences of vertices in the same hyperedges. Little hack here: we make our own. We use our own as the current vectorizer library has a cooccurrence vectorizer based on ordered hyperedges and so has concepts such as "appears before" or "appears after" that we wish to avoid.

The vector representations of the vertices are rows of the weighted adjacency matrix of the hypergraph's 2-section.
From the incidence matrix $H$ (number of vertices x number of hyperedges), this can be obtained via $V_{vectors} = H^TH - D_e$ where $D_e$ is the diagonal matrix that contains the hyperedge sizes. From experiments, we obtain better results by row and column normalizing and reducing the dimensionality of this vertex representation using SVD. Experiments on documents, also show that we get improvements by taking the $4^{th}$ root entry-wise of the normalized matrices before reducing dimensionality. We do the same here because it works well, but it has not been tested on hypergraphs.

In [14]:
def vertexCooccurrenceVectorizer(hyperedges):
    vertexCooccurrence_vectorizer = vectorizers.TokenCooccurrenceVectorizer().fit(recipes)
    vectorizers.TokenCooccurrenceVectorizer(
      token_dictionary=vertexCooccurrence_vectorizer.token_label_dictionary_
    ).fit(hyperedges)
    
    incidence_vectorizer = vectorizers.NgramVectorizer(
    ).fit(hyperedges)

    H = incidence_vectorizer.transform(hyperedges)
    
    M_cooccurrence = (H.T@H)
    M_cooccurrence.setdiag(0)
    M_cooccurrence.eliminate_zeros()
    
    vertexCooccurrence_vectorizer.cooccurrences_ = M_cooccurrence
    
    return(vertexCooccurrence_vectorizer)

In [15]:
%%time

ingredient_vectorizer = vertexCooccurrenceVectorizer(recipes)
ingredient_vectors = ingredient_vectorizer.reduce_dimension(dimension=60, algorithm="randomized")

CPU times: user 9.16 s, sys: 1.21 s, total: 10.4 s
Wall time: 8.32 s


# Build hyperedge vectors

In order to build hyperedge vectors, we will be treating each edge as a distribution over the vertex space. Instead of considering flat distributions of vertices contained in the hyperedges, we use a distribution given by the information weighted incidence matrix. In this incidence matrix, each vertex row gets multiplied by a weight: the information gain of the vertex' observed distribution over hyperedges
$$P_v(e) = \begin{cases}
1/deg(v) &\text{if $v \in e$}\\
0 &\text{otherwise}
\end{cases}
$$  compared to a baseline distribution based of hyperedge sizes $Q(e) = \frac{|e|}{\sum |e|}.$ The weights are computed as follows:

$$\mbox{Info}(v) = \sum_e P_v(e) \log\big(\frac{P_v(e)}{Q(e)}\big).$$
In the case of hyperedges, this will tend to give more weights to vertices that appear in smaller hyperedges and less weights to vertices that appear in all hyperedges.

This is done in two steps: (1) construct the incidence matrix, (2) transform it into its information weighted version.

In [16]:
%%time
incidence_vectorizer = vectorizers.NgramVectorizer(
    token_dictionary=ingredient_vectorizer.token_label_dictionary_
).fit(recipes)

incidence_matrix = incidence_vectorizer.transform(recipes)

CPU times: user 7.36 s, sys: 218 ms, total: 7.58 s
Wall time: 2.59 s


In [17]:
%%time
info_weighted_incidence = vectorizers.transformers.InformationWeightTransformer(
    prior_strength=1e-1,
    approx_prior=False,
).fit_transform(incidence_matrix)

CPU times: user 1.46 s, sys: 19.5 ms, total: 1.48 s
Wall time: 1.46 s


# Joint embedding of vertices and hyperedges

We now have a vector representation for each vertex, and we treat hyperedges as distribution over that space. We can also treat vertices themselves as distributions: a vertex is the Dirac Delta distribution having all mass on its itself (its associated vector).

Now, both vertices and hyperedges are seen as distributions over a vector space. We can now use the Approximate Wasserstein vectorizer. This vectorizer transforms finite distributions over a metric space into vectors in a linear space such that euclidean or cosine distance approximates the Wasserstein distance between the distributions. 

This is done by representing the vertex distributions with the identity matrix on the vertices, and stacking the weighted incidence with this identity matrix. Then give this matrix of both hyperedge and node distributions to the vectorizer function along with the vertex vectors.

In [18]:
n_ingredients = len(ingredient_vectorizer.token_index_dictionary_)

In [19]:
info_doc_with_identity = vstack([info_weighted_incidence, scipy.sparse.identity(n_ingredients)])

In [20]:
%%time
joint_vectors_unsupervised = vectorizers.ApproximateWassersteinVectorizer(
    normalization_power=0.25,
    random_state=42,
).fit_transform(info_doc_with_identity, vectors=ingredient_vectors)

CPU times: user 11.6 s, sys: 1.24 s, total: 12.8 s
Wall time: 464 ms


In [21]:
%%time
joint_vectors_mapper = umap.UMAP(metric="cosine", random_state=42).fit(joint_vectors_unsupervised)

CPU times: user 1min 22s, sys: 53.2 s, total: 2min 15s
Wall time: 34.6 s


# This not that : explore hypergraph

In [22]:
pn.extension()

### Build dataframe that contains information about vertex and hyperedges

In [23]:
n_recipes = len(recipes)
recipes_bool = np.array([True for i in range(n_recipes)] + [False for i in range(n_ingredients)])
ingredients_bool = ~recipes_bool

In [24]:
recipe_metadata_all = pd.DataFrame()
recipe_metadata_all['Type'] = (
                    ['Recipes' for i in range(n_recipes)] 
                       + 
                    ['Ingredients' for x in range(n_ingredients)]
                  )
recipe_metadata_all['Description'] = (
                    [label_name.loc[i]['country'] for i in recipes_label_id.label] 
                       + 
                    [ingredient_vectorizer.token_index_dictionary_[x] for x in range(n_ingredients)]
                  )
recipe_metadata_all['Label'] = (
                    [label_name.loc[i]['new_label'] for i in recipes_label_id.label] 
                       + 
                    ['ingredient' for x in range(n_ingredients)]
                  )
recipe_metadata_all['Ingredients'] = (
                    recipes 
                       + 
                    [ingredient_vectorizer.token_index_dictionary_[x] for x in range(n_ingredients)]
                  )
recipe_metadata_all['Recipe_size'] = (
                    [len(x) for x in recipes] 
                       + 
                    [1 for x in range(n_ingredients)]
                  )

In [25]:
recipe_metadata_all

,Type,Description,Label,Ingredients,Recipe_size
0,Recipes,greek,others.greek,"[romaine lettuce, black olives, grape tomatoes...",9
1,Recipes,southern_us,american.southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",11
2,Recipes,filipino,asian.filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12
3,Recipes,indian,others.indian,"[water, vegetable oil, wheat, salt]",4
4,Recipes,indian,others.indian,"[black pepper, shallots, cornflour, cayenne pe...",20
...,...,...,...,...,...
46263,Ingredients,zesty italian dressing,ingredient,zesty italian dressing,1
46264,Ingredients,zinfandel,ingredient,zinfandel,1
46265,Ingredients,ziti,ingredient,ziti,1
46266,Ingredients,zucchini,ingredient,zucchini,1


In [26]:
# w_plot = recipes_bool | ingredients_bool
w_plot = recipes_bool

In [27]:
recipe_metadata = recipe_metadata_all[w_plot]
recipe_umap = joint_vectors_mapper.embedding_[w_plot]
# recipe_vocab_use_vectors = joint_vectors_unsupervised[w_plot]

In [28]:
layer_metadata = recipe_metadata[['Type', 'Label', 'Recipe_size']].copy()

In [29]:
color_mapping = color_key.copy()
del color_mapping['ingredient']

sizes = [np.sqrt(len(x)) / 100 for x in recipes]

In [30]:
markdown_template = """## Recipe from {Label}
---
#### Ingredients

{Ingredients}

---
"""

### Add a legend

Note: a little tweak to mention is that to control the ordering of the elements in the 

In [31]:
legend = tnt.LegendWidget(
    recipe_metadata.Label,
    factors=list(color_mapping.keys()), 
    palette=list(color_mapping.values()), 
    palette_length=len(color_mapping),
    color_picker_height=16,
    color_picker_margin=[0,0],
    label_height=30,
    label_width=150,
    name="Legend",
    selectable=True,
)

### Search capability
* search_pane

In [32]:
search_pane = tnt.SearchWidget(recipe_metadata, width=400, title="Advanced Search")
# search_pane.link_to_plot(bokeh_plot)

In [33]:
# pn.Row(bokeh_plot, search_pane)

### Summarize selection
* count_summary : count how many things we select
* vertex_summary_pane : list vertices embeded close by

In [34]:
from thisnotthat.summary.dataframe import JointLabelSummarizer, CountSelectedSummarizer
count_summary = tnt.DataSummaryPane(CountSelectedSummarizer(),sizing_mode = "stretch_width")
# count_summary.link_to_plot(bokeh_plot)

In [35]:
vocab = list(recipe_metadata_all.Description[ingredients_bool])
word_summary = JointLabelSummarizer(joint_vectors_unsupervised[recipes_bool],
                                    vocab, 
                                    joint_vectors_unsupervised[ingredients_bool])
vertex_summary_pane = tnt.DataSummaryPane(word_summary)
# vertex_summary_pane.link_to_plot(bokeh_plot)

# Information Pane

In [36]:
info_pane = tnt.InformationPane(recipe_metadata, markdown_template, width=400, height=750, sizing_mode="stretch_height")

In [37]:
#pn.Row(bokeh_plot, count_summary)
#pn.Row(bokeh_plot, pn.Column(count_summary, vertex_summary_pane))

### Add automatic labels

In [40]:
label_layers =  tnt.JointVectorLabelLayers(
    joint_vectors_unsupervised[recipes_bool],      # high dim edge embedding
    recipe_umap,                                   # 2-d edge embedding
    joint_vectors_unsupervised[ingredients_bool],  # high dim vertex embedding
    incidence_vectorizer.column_index_dictionary_, # vertex name
    cluster_map_representation=True,
    min_clusters_in_layer=5,
    random_state=0,
)

In [41]:
annotated_plot = tnt.BokehPlotPane(
    recipe_umap,
    labels=recipe_metadata.Label,
    hover_text=recipe_metadata.Description,
    marker_size=sizes,
    label_color_mapping=color_mapping,
    width=700,
    height=600,
    show_legend=False,
    min_point_size=0.001,
    max_point_size=0.05,
    tools="pan,wheel_zoom,tap,lasso_select,box_zoom,save,reset",
    title="What is cooking? Data Map",
)
annotated_plot.add_cluster_labels(label_layers, max_text_size=24)

In [42]:
count_summary.link_to_plot(annotated_plot)
vertex_summary_pane.link_to_plot(annotated_plot)
search_pane.link_to_plot(annotated_plot)
info_pane.link_to_plot(annotated_plot)
legend.link_to_plot(annotated_plot)

Watcher(inst=LegendWidget(label_color_factors=['asian.chinese', ...], label_color_palette=['#dbe9f6', '#bad6eb', ...], labels=0                others.gr..., name='Legend'), cls=<class 'thisnotthat.label_editor.LegendWidget'>, fn=<function Reactive.link.<locals>.link_cb at 0x7fa0bf997e20>, mode='args', onlychanged=True, parameter_names=('labels', 'label_color_factors', 'label_color_palette'), what='value', queued=False, precedence=0)

In [43]:
pn.Row(annotated_plot, 
       legend,
pn.Tabs(
        pn.Column(count_summary, vertex_summary_pane, name='Selection'),
        search_pane,
        info_pane
    )
)

Row
    [0] BokehPlotPane(color_by_palette=['#dbe9f6', '#bad6eb', ...], color_by_vector=Series([], dtype: object), label_color_factors=['asian.chinese', ...], label_color_palette=['#dbe9f6', '#bad6eb', ...], labels=0                others.gr..., name='Plot')
    [1] LegendWidget(label_color_factors=['asian.chinese', ...], label_color_palette=['#dbe9f6', '#bad6eb', ...], labels=0                others.gr..., name='Legend')
    [2] Tabs
        [0] Column(name='Selection')
            [0] DataSummaryPane(name='Summary')
            [1] DataSummaryPane(name='Summary')
        [1] SearchWidget(data=          Type  Descriptio..., name='Search')
        [2] InformationPane(data=          Type  Descriptio..., name='Information')

In [ ]:
import networkx as nx
import gravis as gv

In [ ]:
selected_edges = [recipes[x] for x in annotated_plot.selected]

In [ ]:
#selected_edges